**Tweet Crawler**

Este caderno é responsável por coletar tweets filtrados por algumas palavras chaves, analisar sentimentos no texto e inserir as coordenadas de onde os tweets foram postados.

Ao fim, um arquivo csv é gerado para analíse dos resultados.

O objetivo geral/motivação foi capturar o sentimento dos postos a respeito da eleição para presidente do Brasil em 28/10/2018, uma segunda parte do trabalho também está disponivel onde pode-se visualizar os resultados (sentimentos) por região. 

Em todo caso, os scripts abaixo podem ser usados em qualquer contexto que se desejar, bastando apenas inserir as chaves da API do Tweet e os termos que se desejam localizar.


In [1]:
'''
Instalando dependências
'''

!pip install textblob==0.15.1
!pip install tweepy==3.6.0
!pip install geocoder

!python -m textblob.download_corpora

    100% |████████████████████████████████| 13.9MB 2.4MB/s 
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
    100% |████████████████████████████████| 634kB 7.1MB/s 
    100% |████████████████████████████████| 286kB 9.9MB/s 
  Running setup.py bdist_wheel for PySocks ... - done
  Stored in directory: /root/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks
    100% |████████████████████████████████| 102kB 4.7MB/s 
    100% |████████████████████████████████| 81kB 5.9MB/s 
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /r

In [0]:
"""
Importando bibliotecas e setando as chaves de acesso a API do tweet
"""

import tweepy

import pandas as pd
from textblob import TextBlob
import geocoder

consumer_key= "CONSUMER_KEY"
consumer_secret= "CONSUMER_SECRET"

access_token= "TWEET_ACCESS_TOKEN"
access_token_secret= "TWEET_ACCESS_TOKEN_SECRET"


''' Setando o acesso ao tweet '''

auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)



In [0]:
"""
Limpando os dados e coletando a geolocalizacao
"""

from time import gmtime, strftime, time


"""
Uma dependencia da biblioteca de análise de sentimentos é que o texto esteja em inglês.
"""
def is_english(text):
    if text.detect_language() == 'en':
        return True
    return False
  
def tweet_analisys(query, ntweets=20):
  """
  Função para criar um dataFrame com os dados coletados, 
  incluindo a latitude e longitude de onde o usuário do tweet estava.
  """

  tweets = tweepy.Cursor(api.search, q="{} -filter:retweets".format(query), result_type="recent", geo_enabled=True).items(ntweets)

  
  df = pd.DataFrame()
  i = 0
  for tweet in tweets:
      phrase = TextBlob(tweet.text)

      if tweet.user.location:
        if not is_english(phrase):
          try:
            phrase = TextBlob(str(phrase.translate(to='en')))
            time.sleep(1)
          except Exception as e:
            print("Um erro na tradução aconteceu na linha {}: {}".format(i,e))
            pass

        df.loc[i, 'texto'] = tweet.text
        df.loc[i, 'polarity'] = str(phrase.sentiment.polarity)
        df.loc[i, 'subjectivity'] = str(phrase.sentiment.subjectivity)
        df.loc[i, 'location'] = tweet.user.location
        g = geocoder.arcgis(df.loc[i, 'location'])
        df.loc[i,'Latitude'] = g.lat
        df.loc[i,'Longitude'] = g.lng
        i = i+1
  
  unique = strftime("%Y%m%d_%H%M%S", gmtime())
  
  print("Criando Extrato aplicado em:", query)
  df.to_csv("{}_{}.csv".format(query,unique), sep=';')
  return df

if __name__ == "__main__":
    [tweet_analisys(q, 3000) for q in ["Bolsonaro", "Haddad"]]
    print("done")
